In [16]:
# Importing necessary libraries
from ipywidgets import VBox, HBox
import ipywidgets as widgets
from IPython.display import clear_output
from PIL import Image
from IPython.display import HTML
from pathlib import Path
import sys
from io import BytesIO
import json
import os
import requests as requests
from IPython.display import HTML
from deepface import DeepFace
from deepface.extendedmodels import  Gender
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from zipfile import ZipFile
import tarfile
from urllib.request import urlopen
from scipy.io import loadmat
import csv
import sys
import re


## Load the gender model from deepface
models = {}
models["gender"] = Gender.loadModel()

In [17]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [18]:
#running deepface
def deepface():
    try:
        deepface_pred = DeepFace.analyze(filename, actions=['gender'], models=models,enforce_detection= True)
        gender= deepface_pred['gender']
        print("Predicted gender for your input is: " + gender)
    except:
         print("Ooops! No face has been detected")

In [19]:
'''Set up layout for upload widgets'''
def initialize_widgets():
    global upload, link, name, layout
    upload = widgets.FileUpload(
        accept='image/*', 
        multiple=False  
    )
    link = widgets.Text(
        placeholder='url to an image file',
        description='Link:',
        disabled=False
    )
 
    input_box = HBox([upload, link])
    layout = VBox([input_box, submit])
    display(layout)
    
global clear, submit, submitted 
submitted = False
print('Please either use upload field to select a file from your computer or provide a url to a web image. ')
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

''' Clear the output and delete the files'''
def on_clear_clicked(b):
    clear_output()
    initialize_widgets()
    for filename in os.listdir('./'):
        if filename.endswith(".jpg") or filename.endswith(".csv"):
            os.remove(filename)
    submitted = False

'''convert images. Save the file (is deleted afterwards)'''    
def preprocess_and_save(content):
    global filename
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    filename= "testfile.jpg"
    image.save(filename)

'''Submitting an image either as fileupload or from url'''
def on_submit_clicked(b):
    global submitted
    global content
    if upload.value and link.value: #both inputs are given
        print('Please choose only one method for input')
        return False
    elif upload.value: #file is uploaded
        uploaded_filename = next(iter(upload.value))
        content = upload.value[uploaded_filename]['content']
    elif link.value: #url is fiven
        content = requests.get(link.value).content
    else: #no input is given
        print('No image has been uploaded')
        return False
    preprocess_and_save(content)
    if submitted: #cleaning the previous results
        clear_output()
        initialize_widgets()
    print('An image you submitted:')
    display(widgets.Image(value=content, width=200))
    deepface()
    display(clear)
    submitted = True

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)



Please either use upload field to select a file from your computer or provide a url to a web image. 


An image you submitted:


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x…

Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

Actions to do:  ['gender']
already built gender model is passed
Ooops! No face has been detected


Button(description='Clear', style=ButtonStyle())

In [20]:
clear_output()
display(HTML('<p>This was a demo version of deepface on one image. You can go to the <a href="deepface_Gender_Prediction_Batch.ipynb" target="_blank">next notebook</a> to apply the model on a batch of images</p>'))